# Data Science Capstone
This notebook will be mainly used for the capstone project.

## Import libraries

In [1]:
import numpy as np

import pandas as pd

from itertools import chain 

import json

import io

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [2]:
# @hidden_cell
foursquare_clientid = "S0N1Q51LLQJU3PBFLDSQHBV2IXDOPL3NTGEUTK02CZ013YKP"
foursquare_clientsecret = "O0H5N2BDLKS3QDO0KQDOBMNI1WFMA4TNTMU4WVHXKW5DO1PA"
version = "20200707"
foursquare_api_str = "https://api.foursquare.com/v2/%s/%s?client_id={}&client_secret={}&v={}".format(
    foursquare_clientid, foursquare_clientsecret, version
)

## Import Toroto neighbourhood data

Import data from Wikipadia, the dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. Add the geographic data.

**Data Cleansing**

Only the datapoints with a no "Not assigned" borough will be cosindered during the analysis. If a datapoint has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.

In [27]:
# neighbourhood data
df_neighborhood = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df_neighborhood = df_neighborhood[df_neighborhood['Borough'] != 'Not assigned']
not_neighborhood_idx = df_neighborhood['Neighborhood'] == 'Not assigned'
df_neighborhood['Neighborhood'][not_neighborhood_idx] = df_neighborhood['Borough'][not_neighborhood_idx]

# # split neiborhood
# def chainer(s):
#     return list(chain.from_iterable(s.str.split(',')))
# lens = df_neighborhood['Neighborhood'].str.split(',').map(len)
# df_neighborhood = pd.DataFrame(
#     {'Postal Code': np.repeat(df_neighborhood['Postal Code'], lens),
#      'Borough': np.repeat(df_neighborhood['Borough'], lens),
#      'Neighborhood': chainer(df_neighborhood['Neighborhood'])}
# )

# geographic data
geo_url = requests.get('https://cocl.us/Geospatial_data').text
df_lat_long = pd.read_csv(io.StringIO(geo_url))

# merge neighbourhood data and geographic data
df_toronto = pd.merge(df_neighborhood, df_lat_long, on='Postal Code')
print('df_toronto.shape: ', df_toronto.shape)
print(df_toronto.head())

df_toronto.shape:  (103, 5)
  Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  
